<a href="https://colab.research.google.com/github/andandandand/practical-computer-vision/blob/main/notebooks/Exploring_Object_Detection_with_FiftyOne_on_COCO_2017.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring Object Detection Performance with FiftyOne

In this notebook we examine the performance of a pretrained [RetinaNet](https://arxiv.org/abs/1708.02002) on the [COCO 2017 validation set](https://cocodataset.org/#home), through the open source [FiftyOne](https://github.com/voxel51/fiftyone) SDK and visualization app.

It covers the following concepts:

- Loading a dataset with ground truth labels [into FiftyOne](https://voxel51.com/docs/fiftyone/user_guide/dataset_creation/index.html)
- [Adding model predictions](https://voxel51.com/docs/fiftyone/recipes/adding_detections.html) to your dataset
- [Evaluating your model](https://voxel51.com/docs/fiftyone/user_guide/evaluation.html#detections) using FiftyOne's evaluation API
- Viewing the best and worst performing samples in your dataset

**So, what's the takeaway?**

Aggregate measures of performance like [mean Average Precision](https://kili-technology.com/data-labeling/machine-learning/mean-average-precision-map-a-complete-guide) don't give us the full picture of your detection model. In practice, the limiting factor on your model's performance is often data quality issues that we need to **see** to address. FiftyOne is designed to make it easy to do just that.

## Inspecting your datasets

Running the workflow presented here on your ML projects will help you to understand the current failure modes (edge cases) of your model and how to fix them, including:

- Identifying scenarios that require additional training samples in order to boost your model's performance
- Deciding whether your ground truth annotations have errors/weaknesses that need to be corrected before any subsequent model training will be profitable


This walkthrough demonstrates how to use FiftyOne to perform hands-on evaluation of your detection model.

## Install FiftyOne

In [124]:
# Install the library, you will need to uncomment this on Colab
#!pip install fiftyone==1.5.2 > /dev/null


In [125]:
#Check installed versions
import fiftyone as fo
print(f"FiftyOne version: {fo.__version__}")

import fiftyone.zoo as foz
import fiftyone.brain as fob # Import common modules once at the top
from fiftyone import ViewField as F

FiftyOne version: 1.5.2


## Dataset loading

In [126]:
dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    dataset_name="evaluate-detections-tutorial",
)
dataset.persistent = True
dataset_clone = dataset.clone()
dataset_clone.persistent = True

print("Dataset loaded.")

# Let's inspect the first sample of the dataset.
# This will show its filepath, tags, metadata, and any existing fields like 'ground_truth'.
print(dataset_clone.first())

INFO:fiftyone.zoo.datasets:Downloading split 'validation' to '/root/fiftyone/coco-2017/validation' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


Images already downloaded


INFO:fiftyone.utils.coco:Images already downloaded


Existing download of split 'validation' is sufficient


INFO:fiftyone.zoo.datasets:Existing download of split 'validation' is sufficient


Loading existing dataset 'evaluate-detections-tutorial'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


INFO:fiftyone.zoo.datasets:Loading existing dataset 'evaluate-detections-tutorial'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


Dataset loaded.
<Sample: {
    'id': '6824a5ae117eced94bbe9951',
    'media_type': 'image',
    'filepath': '/root/fiftyone/coco-2017/validation/data/000000000139.jpg',
    'tags': ['validation'],
    'metadata': <ImageMetadata: {
        'size_bytes': None,
        'mime_type': None,
        'width': 640,
        'height': 426,
        'num_channels': None,
    }>,
    'created_at': datetime.datetime(2025, 5, 14, 15, 3, 18, 171000),
    'last_modified_at': datetime.datetime(2025, 5, 14, 15, 3, 18, 171000),
    'ground_truth': <Detections: {
        'detections': [
            <Detection: {
                'id': '6824a5ae117eced94bbe993d',
                'attributes': {},
                'tags': [],
                'label': 'potted plant',
                'bounding_box': [
                    0.37028125,
                    0.3345305164319249,
                    0.038593749999999996,
                    0.16314553990610328,
                ],
                'mask': None,
           

# Choose a random subset of samples to add predictions to

In [127]:
predictions_view = dataset_clone.take(100, seed=51)

In [128]:
model = foz.load_zoo_model("retinanet-resnet50-fpn-coco-torch")


In [129]:
predictions_view.apply_model(model, label_field="predictions")

 100% |█████████████████| 100/100 [5.6s elapsed, 0s remaining, 18.3 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 100/100 [5.6s elapsed, 0s remaining, 18.3 samples/s]      


In [130]:
session = fo.launch_app(predictions_view, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


INFO:fiftyone.core.session.session:Session launched. Run `session.show()` to open the App in a cell output.


In [131]:
# Shows the predictions on the view only
session.view = predictions_view
print(session.url)

https://5151-gpu-l4-s-2o06qwtjrrtks-a.us-west4-0.prod.colab.dev?polling=true


In [132]:
# Resets the session; the entire dataset will now be shown
session.view = None
print(session.url)

https://5151-gpu-l4-s-2o06qwtjrrtks-a.us-west4-0.prod.colab.dev?polling=true


In [133]:
# Going back to showing the view
session.view = predictions_view
print(session.url)

https://5151-gpu-l4-s-2o06qwtjrrtks-a.us-west4-0.prod.colab.dev?polling=true


Now try inspecting the predictions with confidence > 0.75

![](https://github.com/andandandand/practical-computer-vision/blob/main/images/predictions.png?raw=true)

## Confidence threshold through the SDK

In [134]:
# Only contains detections with confidence >= 0.75
high_conf_view = predictions_view.filter_labels("predictions",
                                                F("confidence") > 0.75,
                                                only_matches=False)

Note the `only_matches=False` argument. When filtering labels, any samples that no longer contain labels would normally be removed from the view. However, this is not desired when performing evaluations since it can skew your results between views. We set `only_matches=False` so that all samples will be retained, even if some no longer contain labels.

In [135]:
# Print some information about the view
print(high_conf_view)

Dataset:     2025.05.14.15.03.18.155643
Media type:  image
Num samples: 100
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    ground_truth:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    predictions:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
View stages:
    1. Take(size=100, seed=51)
    2. FilterLabels(field='predictions', filter={'$gt': ['$$this.confidence', 0.75]}, only_matches=False, trajectories=False)


In [136]:
# Print a prediction from the view to verify that its confidence is > 0.75
sample = high_conf_view.first()
sample

<SampleView: {
    'id': '6824a5b5117eced94bbec3a2',
    'media_type': 'image',
    'filepath': '/root/fiftyone/coco-2017/validation/data/000000151051.jpg',
    'tags': ['validation'],
    'metadata': <ImageMetadata: {
        'size_bytes': None,
        'mime_type': None,
        'width': 640,
        'height': 478,
        'num_channels': None,
    }>,
    'created_at': datetime.datetime(2025, 5, 14, 15, 3, 18, 171000),
    'last_modified_at': datetime.datetime(2025, 5, 14, 15, 3, 18, 655000),
    'ground_truth': <Detections: {
        'detections': [
            <Detection: {
                'id': '6824a5b5117eced94bbec2e2',
                'attributes': {},
                'tags': [],
                'label': 'person',
                'bounding_box': [
                    0.20631249999999998,
                    0.42043933054393307,
                    0.2070625,
                    0.2446234309623431,
                ],
                'mask': None,
                'mask_path': No

In [137]:
# Load high confidence view in the App
session.view = high_conf_view
print(session.url)

https://5151-gpu-l4-s-2o06qwtjrrtks-a.us-west4-0.prod.colab.dev?polling=true


Try inspecting patches on the view

![](https://github.com/andandandand/practical-computer-vision/blob/main/images/inspecting_patches.png?raw=true)

## Evaluate predictions


In [138]:
# Evaluate the predictions in the `faster_rcnn` field of our `high_conf_view`
# with respect to the objects in the `ground_truth` field
results = high_conf_view.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval",
    compute_mAP=True,
)

Evaluating detections...


INFO:fiftyone.utils.eval.detection:Evaluating detections...


 100% |█████████████████| 100/100 [1.4s elapsed, 0s remaining, 58.6 samples/s]          


INFO:eta.core.utils: 100% |█████████████████| 100/100 [1.4s elapsed, 0s remaining, 58.6 samples/s]          


Performing IoU sweep...


INFO:fiftyone.utils.eval.coco:Performing IoU sweep...


 100% |█████████████████| 100/100 [1.1s elapsed, 0s remaining, 84.4 samples/s]          


INFO:eta.core.utils: 100% |█████████████████| 100/100 [1.1s elapsed, 0s remaining, 84.4 samples/s]          


In [139]:
# Get the 10 most common classes in the dataset
counts = dataset_clone.count_values("ground_truth.detections.label")
classes_top10 = sorted(counts, key=counts.get, reverse=True)[:10]

# Print a classification report for the top-10 classes
results.print_report(classes=classes_top10)

               precision    recall  f1-score   support

       person       0.99      0.40      0.57       217
          car       1.00      0.48      0.65        25
        chair       1.00      0.15      0.26        40
         book       0.00      0.00      0.00        52
       bottle       0.50      0.04      0.07        27
          cup       1.00      0.07      0.13        29
 dining table       0.00      0.00      0.00        21
traffic light       0.00      0.00      0.00         1
         bowl       0.00      0.00      0.00        17
      handbag       0.00      0.00      0.00        14

    micro avg       0.98      0.24      0.39       443
    macro avg       0.45      0.11      0.17       443
 weighted avg       0.73      0.24      0.35       443



In [140]:
print(results.mAP())

0.2794843881248003


In [141]:
# install ipywidgets if needed for the interactive plot below
#!pip install 'ipywidgets>=8,<9'

In [142]:
plot = results.plot_pr_curves(classes=["person", "car"])
plot.show()

## Sample level analysis

In [143]:
# Our dataset's schema now contains `eval_*` fields from a confusion matrix
print(dataset_clone)

Name:        2025.05.14.15.03.18.155643
Media type:  image
Num samples: 5000
Persistent:  True
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    ground_truth:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    predictions:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    eval_tp:          fiftyone.core.fields.IntField
    eval_fp:          fiftyone.core.fields.IntField
    eval_fn:          fiftyone.core.fields.IntField


In [144]:
# The dataset keeps track of the evaluations that we have run
print(dataset_clone.list_evaluations())

['eval']


In [145]:
print(dataset_clone.get_evaluation_info("eval"))

{
    "key": "eval",
    "version": "1.5.2",
    "timestamp": "2025-05-14T15:03:33.136000",
    "config": {
        "cls": "fiftyone.utils.eval.coco.COCOEvaluationConfig",
        "type": "detection",
        "method": "coco",
        "pred_field": "predictions",
        "gt_field": "ground_truth",
        "iou": 0.5,
        "classwise": true,
        "custom_metrics": null,
        "iscrowd": "iscrowd",
        "use_masks": false,
        "use_boxes": false,
        "tolerance": null,
        "compute_mAP": true,
        "iou_threshs": [
            0.5,
            0.55,
            0.6,
            0.65,
            0.7,
            0.75,
            0.8,
            0.85,
            0.9,
            0.95
        ],
        "max_preds": 100,
        "error_level": 1
    }
}


In [146]:
# Load the view on which we ran the `eval` evaluation
eval_view = dataset_clone.load_evaluation_view("eval")
print(eval_view)

Dataset:     2025.05.14.15.03.18.155643
Media type:  image
Num samples: 100
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    ground_truth:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    predictions:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    eval_tp:          fiftyone.core.fields.IntField
    eval_fp:          fiftyone.core.fields.IntField
    eval_fn:          fiftyone.core.fields.IntField
View stages:
    1. Take(size=100, seed=51)
    2. FilterLabels(field='predictions', filter={'$gt': ['$$this.confidence', 0.75]}, only_matche

In [147]:
# Our detections have helpful evaluation data on them
sample = high_conf_view.first()
print(sample.predictions.detections[0])

<Detection: {
    'id': '6824b0b6117eced94bbf5ce5',
    'attributes': {},
    'tags': [],
    'label': 'person',
    'bounding_box': [
        0.21051760017871857,
        0.4170771539211273,
        0.20937219262123108,
        0.27810138463974,
    ],
    'mask': None,
    'mask_path': None,
    'confidence': 0.9692026376724243,
    'index': None,
    'eval_iou': 0.8374545505080747,
    'eval_id': '6824a5b5117eced94bbec2e2',
    'eval': 'tp',
}>


In [148]:
# View the `iscrowd` attribute on a ground truth object
sample = dataset_clone.first()
print(sample.ground_truth.detections[0])

<Detection: {
    'id': '6824a5ae117eced94bbe993d',
    'attributes': {},
    'tags': [],
    'label': 'potted plant',
    'bounding_box': [
        0.37028125,
        0.3345305164319249,
        0.038593749999999996,
        0.16314553990610328,
    ],
    'mask': None,
    'mask_path': None,
    'confidence': None,
    'index': None,
    'supercategory': 'furniture',
    'iscrowd': 0,
}>


## Evaluation patches

So, now that we have a sense for the aggregate performance of our model, let's dive into sample-level analysis by creating an [evaluation view](https://voxel51.com/docs/fiftyone/user_guide/app.html#viewing-evaluation-patches).

Any evaluation that you stored on your dataset can be used to generate an [evaluation view](https://voxel51.com/docs/fiftyone/user_guide/app.html#viewing-evaluation-patches) that is a patches view creating a sample for every true positive, false positive, and false negative in your dataset.
Through this view, you can quickly filter and sort evaluated detections by their type (TP/FP/FN), evaluated IoU, and if they are matched to a crowd object.

These evaluation views can be created through Python or directly in the App as shown below.

In [149]:
eval_patches = dataset_clone.to_evaluation_patches("eval")
print(eval_patches)

Dataset:     2025.05.14.15.03.18.155643
Media type:  image
Num patches: 37593
Patch fields:
    id:               fiftyone.core.fields.ObjectIdField
    sample_id:        fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    ground_truth:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    predictions:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    crowd:            fiftyone.core.fields.BooleanField
    type:             fiftyone.core.fields.StringField
    iou:              fiftyone.core.fields.FloatField
View stages:
    1. ToEvaluationPatches(eval_key='eval', config=None)


In [150]:
# let's use this evaluation to find false positives with confidence above > .85
session_view = high_conf_view
print(session.url )

https://5151-gpu-l4-s-2o06qwtjrrtks-a.us-west4-0.prod.colab.dev?polling=true


![](https://github.com/andandandand/practical-computer-vision/blob/main/images/false_positive.png?raw=true)

## View with best performing cases

In [151]:
# Show samples with most true positives
session.view = high_conf_view.sort_by("eval_tp", reverse=True)
print(session.url)

https://5151-gpu-l4-s-2o06qwtjrrtks-a.us-west4-0.prod.colab.dev?polling=true


## View with the worst performing cases by false positives

In [152]:
# Show samples with most false positives
session.view = high_conf_view.sort_by("eval_fp", reverse=True)
print(session.url)

https://5151-gpu-l4-s-2o06qwtjrrtks-a.us-west4-0.prod.colab.dev?polling=true


## View with the best performing cases by false negatives

In [153]:
# Show samples with most false negatives
session.view = high_conf_view.sort_by("eval_fn", reverse=True)
print(session.url)

https://5151-gpu-l4-s-2o06qwtjrrtks-a.us-west4-0.prod.colab.dev?polling=true


## Filtering by bounding box area

In [154]:
# Compute metadata so we can reference image height/width in our view
dataset.compute_metadata()

In [155]:
#
# Create an expression that will match objects whose bounding boxes have
# area less than 32^2 pixels
#
# Bounding box format is [top-left-x, top-left-y, width, height]
# with relative coordinates in [0, 1], so we multiply by image
# dimensions to get pixel area
#
bbox_area = (
    F("$metadata.width") * F("bounding_box")[2] *
    F("$metadata.height") * F("bounding_box")[3]
)
small_boxes = bbox_area < 32 ** 2

# Create a view that contains only small (and high confidence) predictions
small_boxes_view = high_conf_view.filter_labels("predictions", small_boxes)

session.view = small_boxes_view
print(session.url)

https://5151-gpu-l4-s-2o06qwtjrrtks-a.us-west4-0.prod.colab.dev?polling=true


In [156]:
# Create a view that contains only small GT and predicted boxes
small_boxes_eval_view = (
    high_conf_view
    .filter_labels("ground_truth", small_boxes, only_matches=False)
    .filter_labels("predictions", small_boxes, only_matches=False)
)

# Run evaluation
small_boxes_results = small_boxes_eval_view.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
)

Evaluating detections...


INFO:fiftyone.utils.eval.detection:Evaluating detections...


 100% |█████████████████| 100/100 [238.5ms elapsed, 0s remaining, 419.3 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 100/100 [238.5ms elapsed, 0s remaining, 419.3 samples/s]      


In [157]:
# Get the 10 most common small object classes
small_counts = small_boxes_eval_view.count_values("ground_truth.detections.label")
classes_top10_small = sorted(small_counts, key=counts.get, reverse=True)[:10]

# Print a classification report for the top-10 small object classes
small_boxes_results.print_report(classes=classes_top10_small)

               precision    recall  f1-score   support

       person       1.00      0.04      0.08        47
          car       0.00      0.00      0.00         6
        chair       0.00      0.00      0.00         7
         book       0.00      0.00      0.00        32
       bottle       0.00      0.00      0.00        13
          cup       0.00      0.00      0.00        16
 dining table       0.00      0.00      0.00         1
traffic light       0.00      0.00      0.00         1
         bowl       0.00      0.00      0.00         7
      handbag       0.00      0.00      0.00         3

    micro avg       1.00      0.02      0.03       133
    macro avg       0.10      0.00      0.01       133
 weighted avg       0.35      0.02      0.03       133



## Inspecting the crowd views

In [158]:
# View the `iscrowd` attribute on a ground truth object
sample = dataset_clone.first()
print(sample.ground_truth.detections[0])

<Detection: {
    'id': '6824a5ae117eced94bbe993d',
    'attributes': {},
    'tags': [],
    'label': 'potted plant',
    'bounding_box': [
        0.37028125,
        0.3345305164319249,
        0.038593749999999996,
        0.16314553990610328,
    ],
    'mask': None,
    'mask_path': None,
    'confidence': None,
    'index': None,
    'supercategory': 'furniture',
    'iscrowd': 0,
}>


In [159]:
# Create a view that contains only samples for which at least one detection has
# its iscrowd attribute set to 1
crowded_images_view = high_conf_view.match(
    F("ground_truth.detections").filter(F("iscrowd") == 1).length() > 0
)

session.view = crowded_images_view
print(session.url)

https://5151-gpu-l4-s-2o06qwtjrrtks-a.us-west4-0.prod.colab.dev?polling=true


In [160]:
# Evaluating the crowd by the number of false positives
session.view = crowded_images_view.sort_by("eval_fp", reverse=True)
print(session.url)

https://5151-gpu-l4-s-2o06qwtjrrtks-a.us-west4-0.prod.colab.dev?polling=true


## Using the model to improve the dataset (active learning)

In [162]:
# Tag all highly confident false positives as "possibly-missing"
(
    high_conf_view
        .filter_labels("predictions", F("eval") == "fp")
        .select_fields("predictions")
        .tag_labels("possibly-missing")
)

These tagged labels could then be sent off to our annotation provider of choice for review and addition to the ground truth labels. FiftyOne currently offers integrations for [Scale AI](https://voxel51.com/docs/fiftyone/api/fiftyone.utils.scale.html), [Labelbox](https://voxel51.com/docs/fiftyone/api/fiftyone.utils.labelbox.html), and [CVAT](https://voxel51.com/docs/fiftyone/api/fiftyone.types.dataset_types.html?highlight=cvat#fiftyone.types.dataset_types.CVATImageDataset).


In [163]:
# Export all labels with the `possibly-missing` tag in CVAT format
(
    dataset
        .select_labels(tags=["possibly-missing"])
        .export("./possibly_missing_labels", fo.types.CVATImageDataset)
)

 100% |█████████████████████| 0/0 [32.9ms elapsed, ? remaining, ? samples/s] 


INFO:eta.core.utils: 100% |█████████████████████| 0/0 [32.9ms elapsed, ? remaining, ? samples/s] 


## Summary

In this tutorial, we covered loading a dataset into FiftyOne and analyzing the performance of an out-of-the-box object detection model on the dataset.

**So, what's the takeaway?**

Aggregate evaluation results for an object detector are important, but they alone don't tell the whole story of a model's performance. It's critical to study the failure modes of your model so you can take the right actions to improve them.

In this tutorial, we covered two types of analysis:

- Analyzing the performance of your detector across different strata, like high confidence, small objects in crowded scenes
- Inspecting the hardest samples in your dataset to diagnose the underlying issue, whether it be your detector or the ground truth annotations

## About this tutorial

This tutorial is based on [FiftyOne's documentation](https://docs.voxel51.com/tutorials/evaluate_detections.html). You will notice a couple of minor changes.

* Views for the app launch on their own window, this makes it easier for us to inspect the output of our views on the app.
* We create a clone of the COCO dataset at the start of the notebook so that we can go back to its original state if we want
* I replaced Faster R-CNN for RetinaNet. As an exercise I encourage to try it with [Faster R-CNN from our model zoo](https://docs.voxel51.com/model_zoo/models.html#faster-rcnn-resnet50-fpn-coco-torch) or through our [integration with Ultralytics's YOLO](https://docs.voxel51.com/integrations/ultralytics.html).

